## Машинное обучение

*Подгрузим библиотеки, которые нам понадобятся, и файл*

In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_percentage_error

import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv("data/auto_new_.csv")
df.head(5)

,model auto,year of production,mileage,exterior colour,tax,transmission,wheel drive,number of owners,engine volume,engine power,...,acceleration,MPG,country of the car brand,class,top speed,environmental class,price,fuel utilization efficiency,engine optimality,intensity of use
0,Mercedes-Benz,2019,98500.0,чёрный,55050.0,автоматическая,полный,1 владелец,3.0,367.0,...,6.2,9.4,Германия,F,246.0,Euro 6,9299000.0,3.133333,6826.20,16416.666667
1,Mercedes-Benz,2018,51400.0,чёрный,4760.0,автоматическая,задний,2 владельца,2.1,136.0,...,12.8,5.8,Германия,M,181.0,Euro 5,12800000.0,2.761905,3655.68,7342.857143
2,Mercedes-Benz,2020,23500.0,чёрный,18675.0,автоматическая,полный,1 владелец,2.9,249.0,...,7.4,9.8,Германия,J,199.0,Euro 6,17499000.0,3.379310,5343.54,4700.000000
3,BMW,2021,50300.0,чёрный,51000.0,автоматическая,полный,2 владельца,3.0,340.0,...,5.5,8.6,Германия,E,250.0,Euro 6,9990000.0,2.866667,5610.00,12575.000000
4,BMW,2022,6152.0,серый,51000.0,автоматическая,полный,1 владелец,3.0,340.0,...,4.6,5.1,Германия,E,250.0,Euro 6,9900000.0,1.700000,4692.00,2050.666667


*Для начала столбцы, которые являются категориальными, преобразуем с помощью OHE, задав удаление первого значения, чтобы избежать образования линейной зависимости*

In [3]:
X = df[['model auto', 'year of production', 'mileage', 'exterior colour', 'tax',
       'transmission', 'wheel drive', 'number of owners', 'engine volume',
       'engine power', 'engine type', 'fuel type', 'acceleration', 'MPG',
       'country of the car brand', 'class', 'top speed', 'environmental class',
       'fuel utilization efficiency', 'engine optimality', 'intensity of use']]

y = df['price']

cat_features_mask = (X.dtypes == "object").values
X_cat = X[X.columns[cat_features_mask]]

X_ohe = pd.get_dummies(X, columns = X_cat.columns, drop_first = True)

X_ohe

,year of production,mileage,tax,engine volume,engine power,acceleration,MPG,top speed,fuel utilization efficiency,engine optimality,...,class_E,class_F,class_J,class_M,class_S,environmental class_Euro 2,environmental class_Euro 3,environmental class_Euro 4,environmental class_Euro 5,environmental class_Euro 6
0,2019,98500.0,55050.0,3.0,367.0,6.2,9.4,246.0,3.133333,6826.20,...,False,True,False,False,False,False,False,False,False,True
1,2018,51400.0,4760.0,2.1,136.0,12.8,5.8,181.0,2.761905,3655.68,...,False,False,False,True,False,False,False,False,True,False
2,2020,23500.0,18675.0,2.9,249.0,7.4,9.8,199.0,3.379310,5343.54,...,False,False,True,False,False,False,False,False,False,True
3,2021,50300.0,51000.0,3.0,340.0,5.5,8.6,250.0,2.866667,5610.00,...,True,False,False,False,False,False,False,False,False,True
4,2022,6152.0,51000.0,3.0,340.0,4.6,5.1,250.0,1.700000,4692.00,...,True,False,False,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2568,2018,81244.0,3520.0,1.6,110.0,10.4,5.8,191.0,3.625000,1830.40,...,False,False,False,False,False,False,False,False,True,False
2569,2015,159000.0,17430.0,3.0,249.0,6.7,6.0,230.0,2.000000,5004.90,...,True,False,False,False,False,False,False,False,False,True
2570,2010,288910.0,9120.0,2.7,190.0,12.7,10.2,180.0,3.777778,6515.10,...,False,False,True,False,False,False,False,False,True,False
2571,2007,203000.0,3360.0,1.6,112.0,11.5,7.5,197.0,4.687500,2060.80,...,False,False,False,False,False,False,False,False,False,False


*Создадим функцию для вывода ошибки модели по разным функциям потерь*

In [4]:
def result_mod(R2, RMSE, MAPE):
    print(f"R2 = {R2}, RMSE = {RMSE}, MAPE = {MAPE}")

*Для начала разделим выборку на тренировочную, валидационную и тестовую*

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X_ohe, y, random_state = 42, test_size=0.3)

*Теперь воспользуемся RobustScaler, так как он неплохо справляется с выбросами*

In [6]:
scaler = RobustScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

*Определим функциию потерь: RMSE*

In [7]:
def RMSE(y_test, y_pred):
    return np.sqrt((y_test - y_pred) ** 2).mean()

*Теперь, когда наши данные готовы, попробуем обучить для начала kNN и посмотрим на ошибки, которые получаются*

In [8]:
k = np.arange(1, 101, 1)
searcher = GridSearchCV(
    KNeighborsRegressor(), [{"n_neighbors": k}],
    scoring = "neg_root_mean_squared_error", cv = 10
)

searcher.fit(X_train, y_train)

best_n_neighbors = searcher.best_params_["n_neighbors"]

final_model_kNN = KNeighborsRegressor(n_neighbors = best_n_neighbors)
final_model_kNN.fit(X_train, y_train)
y_pred = final_model_kNN.predict(X_test)

result_mod(r2_score(y_test, y_pred), RMSE(y_test, y_pred), mean_absolute_percentage_error(y_test, y_pred))

R2 = 0.8731267006288224, RMSE = 679436.207253886, MAPE = 0.22481733942662255


*Данная модель показала себя достаточно неплохо, имея ошибку по RMSE около 700 тысяч рублей*

*Попробуем обучить линейную регрессию, посмотрим, какие у неё значения коэффициентов модели и ошибка*

In [9]:
model_lin = LinearRegression()
model_lin.fit(X_train, y_train)
y_pred = model_lin.predict(X_test)

result_mod(r2_score(y_test, y_pred), RMSE(y_test, y_pred), mean_absolute_percentage_error(y_test, y_pred))

model_lin.coef_

R2 = -1.9983093069467966e+20, RMSE = 3378775297868209.5, MAPE = 21577810762.87515


array([ 2.71188464e+06,  1.01360909e+06,  4.80283510e+05, -1.99414536e+06,
        1.58183589e+06, -3.21946156e+05,  8.71849847e+05, -4.07805562e+05,
       -9.27413288e+05,  4.86039639e+05, -9.21517438e+05,  6.63895351e+05,
        1.45483139e+18,  2.29468214e+18, -3.37279686e+18, -3.37279686e+18,
        2.29468214e+18, -1.07992149e+18,  1.21565322e+18, -6.64804890e+17,
        5.94840049e+16, -3.37279686e+18, -3.37279686e+18,  1.35526381e+18,
        2.29468214e+18, -3.37279686e+18,  6.38107927e+17, -3.37279686e+18,
        5.94840049e+16,  6.38107927e+17,  5.94840049e+16,  5.94840049e+16,
        1.45483139e+18,  2.29468214e+18,  6.38107927e+17,  1.21565322e+18,
        1.35526381e+18,  1.45483139e+18,  5.94840049e+16, -3.37279686e+18,
       -3.37279686e+18,  1.35526381e+18,  5.94840049e+16,  1.59296000e+06,
        1.45483139e+18,  5.94840049e+16,  5.94840049e+16, -4.18048000e+05,
       -1.07992149e+18, -7.56352000e+05, -1.07992149e+18,  6.35686916e+17,
        6.38107927e+17,  

*Видим, что мы получили огромную ошибку и большие коэффициенты модели, что может свидетельствовать о переобучении модели*

In [10]:
alphas = np.arange(0, 1000, 10)
searcher = GridSearchCV(
    Ridge(), [{"alpha": alphas}],
    scoring = "neg_root_mean_squared_error", cv = 10
)

searcher.fit(X_train, y_train)

best_alpha = searcher.best_params_["alpha"]

final_model_ridge = Ridge(alpha = best_alpha, random_state = 42)
final_model_ridge.fit(X_train, y_train)

y_pred = final_model_ridge.predict(X_test)

result_mod(r2_score(y_test, y_pred), RMSE(y_test, y_pred), mean_absolute_percentage_error(y_test, y_pred))

R2 = 0.7612343674176565, RMSE = 1237360.5140495705, MAPE = 0.672764973522591


In [11]:
alphas = np.arange(0, 1000, 10)
searcher = GridSearchCV(
    Lasso(), [{"alpha": alphas}],
    scoring = "neg_root_mean_squared_error", cv = 10
)

searcher.fit(X_train, y_train)

best_alpha = searcher.best_params_["alpha"]

final_model_ridge = Lasso(alpha = best_alpha, random_state = 42)
final_model_ridge.fit(X_train, y_train)

y_pred = final_model_ridge.predict(X_test)

result_mod(r2_score(y_test, y_pred), RMSE(y_test, y_pred), mean_absolute_percentage_error(y_test, y_pred))

R2 = 0.7742665904245625, RMSE = 1235452.5596984662, MAPE = 0.7111625232949167


*Делаем вывод, что Ridge и Lasso регрессии улучшили результаты линейной регрессии, но какая-то одна модель не выделяется лучшим результатом*

In [12]:
model_forest = RandomForestRegressor(n_estimators = 1000, random_state = 42)
model_forest.fit(X_train, y_train)

y_pred = model_forest.predict(X_test)

result_mod(r2_score(y_test, y_pred), RMSE(y_test, y_pred), mean_absolute_percentage_error(y_test, y_pred))

R2 = 0.9478548407933057, RMSE = 455915.1971397113, MAPE = 0.171432599143094


*Таким образом, лучшая модель, которую нам получилось построить, - это случайный лес, который дал нам среднеквадратичную ошибку в 455915 рублей*

*Исходя из представленной информации, можно сделать вывод о высокой точности модели случайного леса. Значение коэффициента детерминации (R2) идентично значению точности модели и составляет 0.9478548407933057. Это означает, что около 94.8% вариации зависимой переменной объясняется моделью, что является очень хорошим показателем.*

*Также рассчитаны значения среднеквадратичной ошибки (RMSE) и средней абсолютной процентной ошибки (MAPE). RMSE равна 455915.1971397113, что означает, что средняя ошибка прогнозирования составляет примерно 455915 рублей. MAPE равна 0.171432599143094, что указывает на то, что средняя абсолютная процентная ошибка составляет примерно 17.1%. Это свидетельствует о хорошем качестве прогнозов модели.*

*Таким образом, можно сделать вывод о том, что модель случайного леса обладает высокой точностью и предсказывает целевую переменную с небольшой средней ошибкой*